In [1]:
import warnings
warnings.filterwarnings('ignore')
#warnings.filterwarnings(action='once')

In [5]:
import numpy as np
import numpy.testing as npt
#from fixtures import dtlz2_5x_3f_data_based  # noqa: F401


import polars as pl

import pandas as pd


from desdeo.mcdm.nautili import (
    calculate_distance_to_front,
    calculate_navigation_point,
    solve_reachable_bounds,
    solve_reachable_solution,
    agg_maxmin, 
    agg_maxmin_cones,
    nautili_all_steps,
    NAUTILI_Response, 
    aggregate,
    nautili_all_steps,
    nautili_init
)

from desdeo.problem import (
    binh_and_korn,
    momip_ti7,
    objective_dict_to_numpy_array,
    river_pollution_problem,
    get_nadir_dict,
    get_ideal_dict,
    dtlz2,
    #variable_dict_to_numpy_array,
    objective_dict_to_numpy_array
)
from desdeo.tools import (
    BonminOptions,
    NevergradGenericOptions,
    NevergradGenericSolver,
    PyomoBonminSolver,
    ScipyMinimizeSolver,
)


#def test_dtzl2(pref_agg_method):
    #problem = river_pollution_problem
problem = dtlz2(10,3)
nadir = get_nadir_dict(problem)
ideal = get_ideal_dict(problem)
total_steps = 5
pref_agg_method="maxmin_cones"
DMs = 4

initial_response = nautili_init(problem)
rps = {
    "DM1": {"f_1": 0.5, "f_2": 0.8, "f_3": 0.7}, # dm1 and dm2 prefer f1
    "DM2": {"f_1": 0.5, "f_2": 0.7, "f_3": 0.8},
    "DM3": {"f_1": 0.5, "f_2": 0.3, "f_3": 0.5},
    "DM4": {"f_1": 0.2, "f_2": 0.5, "f_3": 0.5},

}
all_resp = nautili_all_steps(
    problem,
    total_steps,
    rps,
    [initial_response], # Note that this is a list of NAUTILUS_Response objects
    pref_agg_method=pref_agg_method, # used pref agg method
)
#print(all_resp)
print(all_resp[-1].reference_points)
print(all_resp[-1].group_improvement_direction)
print(all_resp[-1].navigation_point)
print("reachable solution:", all_resp[-1].reachable_solution)

#return all_resp

maxmin_cones
number of Objs:3, number of DMs:4
`gtol` termination condition is satisfied.
Number of iterations: 32, function evaluations: 162, CG iterations: 50, optimality: 8.82e-13, constraint violation: 2.22e-16, execution time: 0.062 s.
[0.45317009 0.63881718 0.6724232 ]
{'DM1': {'f_1': 0.5, 'f_2': 0.8, 'f_3': 0.7}, 'DM2': {'f_1': 0.5, 'f_2': 0.7, 'f_3': 0.8}, 'DM3': {'f_1': 0.5, 'f_2': 0.3, 'f_3': 0.5}, 'DM4': {'f_1': 0.2, 'f_2': 0.5, 'f_3': 0.5}}
None
{'f_1': 0.4851436639748217, 'f_2': 0.6672148372697848, 'f_3': 0.7005276426385306}
reachable solution: {'f_1': 0.4851436639748217, 'f_2': 0.6672148372697848, 'f_3': 0.7005276426385306}


In [8]:
problem2 = momip_ti7() # does not work for some reason
nadir = get_nadir_dict(problem2)
ideal = get_ideal_dict(problem2)
print(nadir)
print(ideal)
# f1 min, f2 max, f3 min

{'f_1': 3.0, 'f_2': -3.0, 'f_3': 3.0}
{'f_1': -3.0, 'f_2': 3.0, 'f_3': -3.0}


In [14]:
problem2 = binh_and_korn() # does not work for some reason
nadir = get_nadir_dict(problem2)
ideal = get_ideal_dict(problem2)
print(nadir)
print(ideal)

{'f_1': 140.0, 'f_2': 50.0}
{'f_1': 0.0, 'f_2': 0.0}


In [3]:
total_steps = 5
steps_remaining = 1
## take a step back
pref_agg_method="maxmin_cones"

all_resp_one = all_resp[:4]
# Navigation point: {'f_1': 1.0166947028186726, 'f_2': 0.9994040585920181, 'f_3': 0.6028379478090141}
rps = {
    "DM1": {"f_1": 0.8, "f_2": 0.5, "f_3": 0.5}, # dm1 and dm2 prefer f1
    "DM2": {"f_1": 0.62, "f_2": 0.62, "f_3": 0.58},
    "DM3": {"f_1": 0.5, "f_2": 0.7, "f_3": 0.53},
    "DM4": {"f_1": 0.3, "f_2": 0.3, "f_3": 0.15},
    "DM5": {"f_1": 0.2, "f_2": 0.5, "f_3": 0.25},
    #"DM4": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
}
all_resp_new = nautili_all_steps(
    problem,
    steps_remaining,
    rps,
    all_resp_one, # Note that this is a list of NAUTILUS_Response objects
    pref_agg_method=pref_agg_method, # used pref agg method
)
#print(all_resp)
print(all_resp_new[-1].reference_points)
print(all_resp_new[-1].group_improvement_direction)
print(all_resp_new[-1].navigation_point)
print("reachable solution:", all_resp_new[-1].reachable_solution)

maxmin_cones
number of Objs:3, number of DMs:5
`gtol` termination condition is satisfied.
Number of iterations: 37, function evaluations: 210, CG iterations: 68, optimality: 2.31e-13, constraint violation: 4.44e-16, execution time: 0.095 s.
[0.58870359 0.55494609 0.47245054]
{'DM1': {'f_1': 0.8, 'f_2': 0.5, 'f_3': 0.5}, 'DM2': {'f_1': 0.62, 'f_2': 0.62, 'f_3': 0.58}, 'DM3': {'f_1': 0.5, 'f_2': 0.7, 'f_3': 0.53}, 'DM4': {'f_1': 0.3, 'f_2': 0.3, 'f_3': 0.15}, 'DM5': {'f_1': 0.2, 'f_2': 0.5, 'f_3': 0.25}}
{'f_1': 0.4279911095404503, 'f_2': 0.4444579683260891, 'f_3': 0.13038740812972682}
{'f_1': 3.657132581846952e-29, 'f_2': -1.04744440165294e-14, 'f_3': 3.0}
reachable solution: {'f_1': 3.657132581846952e-29, 'f_2': -1.04744440165294e-14, 'f_3': 3.0}


In [4]:
all_resp_new

[NAUTILI_Response(step_number=5, distance_to_front=100.0, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.5, 'f_3': 0.5}, 'DM2': {'f_1': 0.62, 'f_2': 0.62, 'f_3': 0.58}, 'DM3': {'f_1': 0.5, 'f_2': 0.7, 'f_3': 0.53}, 'DM4': {'f_1': 0.3, 'f_2': 0.3, 'f_3': 0.15}, 'DM5': {'f_1': 0.2, 'f_2': 0.5, 'f_3': 0.25}}, improvement_directions={'DM1': array([0.2166947 , 0.49940406, 0.10283795]), 'DM2': array([0.3966947 , 0.37940406, 0.02283795]), 'DM3': array([0.5166947 , 0.29940406, 0.07283795]), 'DM4': array([0.7166947 , 0.69940406, 0.45283795]), 'DM5': array([0.8166947 , 0.49940406, 0.35283795])}, group_improvement_direction={'f_1': 0.4279911095404503, 'f_2': 0.4444579683260891, 'f_3': 0.13038740812972682}, group_reference_point={'f_1': 0.5887035932782223, 'f_2': 0.554946090265929, 'f_3': 0.4724505396792873}, navigation_point={'f_1': 3.657132581846952e-29, 'f_2': -1.04744440165294e-14, 'f_3': 3.0}, reachable_solution={'f_1': 3.657132581846952e-29, 'f_2': -1.04744440165294e-14, 'f_3': 3.0}, reachab

In [7]:
all_resp

[NAUTILI_Response(step_number=1, distance_to_front=20.0, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.4, 'f_3': 0.7}, 'DM2': {'f_1': 0.62, 'f_2': 0.62, 'f_3': 0.8}, 'DM3': {'f_1': 0.5, 'f_2': 0.7, 'f_3': 0.53}, 'DM4': {'f_1': 0.3, 'f_2': 0.3, 'f_3': 0.15}, 'DM5': {'f_1': 0.2, 'f_2': 0.5, 'f_3': 0.25}}, improvement_directions={'DM1': array([1.2, 1.6, 1.3]), 'DM2': array([1.38, 1.38, 1.2 ]), 'DM3': array([1.5 , 1.3 , 1.47]), 'DM4': array([1.7 , 1.7 , 1.85]), 'DM5': array([1.8 , 1.5 , 1.75])}, group_improvement_direction={'f_1': 1.4538189625976647, 'f_2': 1.480553318429934, 'f_3': 1.4444960861412373}, group_reference_point={'f_1': 0.5461810374023354, 'f_2': 0.519446681570066, 'f_3': 0.5555039138587626}, navigation_point={'f_1': 1.7541736757046682, 'f_2': 1.7498510146480046, 'f_3': 1.6507094869522536}, reachable_solution={'f_1': 0.7708683785233409, 'f_2': 0.7492550732400227, 'f_3': 0.2535474347612677}, reachable_bounds={'lower_bounds': {'f_1': 4.6347885317295557e-05, 'f_2': 0.0, 'f_3': 5

In [3]:
## take a step back

#nadir = get_nadir_dict(problem)
#ideal = get_ideal_dict(problem)

# S1 prefs
rps = {
    "DM1": {"f_1": 0.5, "f_2": 0.9, "f_3": 0.9},
    "DM2": {"f_1": 0.9, "f_2": 0.5, "f_3": 0.9},
    "DM3": {"f_1": 0.9, "f_2": 0.9, "f_3": 0.5},
    #"DM4": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
}

rps = {
    "DM1": {"f_1": 0.8, "f_2": 0.7, "f_3": 0.6},
    "DM2": {"f_1": 0.7, "f_2": 0.8, "f_3": 0.5},
    "DM3": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
    #"DM4": {"f_1": 0.5, "f_2": 0.6, "f_3": 0.8},
}
#print("boiunds",b)rps, [initial_response], pref_agg_method)

SyntaxError: unmatched ')' (3398968460.py, line 10)

In [32]:
all_resp

[NAUTILI_Response(step_number=1, distance_to_front=20.0, reference_points={'DM1': {'f_1': 0.8, 'f_2': 0.4, 'f_3': 0.7}, 'DM2': {'f_1': 0.62, 'f_2': 0.62, 'f_3': 0.8}, 'DM3': {'f_1': 0.5, 'f_2': 0.7, 'f_3': 0.53}, 'DM4': {'f_1': 0.3, 'f_2': 0.3, 'f_3': 0.15}, 'DM5': {'f_1': 0.2, 'f_2': 0.5, 'f_3': 0.25}}, improvement_directions={'DM1': array([1.2, 1.6, 1.3]), 'DM2': array([1.38, 1.38, 1.2 ]), 'DM3': array([1.5 , 1.3 , 1.47]), 'DM4': array([1.7 , 1.7 , 1.85]), 'DM5': array([1.8 , 1.5 , 1.75])}, group_improvement_direction={'f_1': 1.4538189625976647, 'f_2': 1.480553318429934, 'f_3': 1.4444960861412373}, group_reference_point={'f_1': 0.5461810374023354, 'f_2': 0.519446681570066, 'f_3': 0.5555039138587626}, navigation_point={'f_1': 1.7541736757046682, 'f_2': 1.7498510146480046, 'f_3': 1.6507094869522536}, reachable_solution={'f_1': 0.7708683785233409, 'f_2': 0.7492550732400227, 'f_3': 0.2535474347612677}, reachable_bounds={'lower_bounds': {'f_1': 4.6347885317295557e-05, 'f_2': 0.0, 'f_3': 5

In [5]:
all_resp[-1].reachable_solution

{'f_1': 0.6062778747202097,
 'f_2': 0.6066241719195791,
 'f_3': 0.6087682451937068}

In [6]:
all_resp[0].group_improvement_direction

{'f_1': 1.9999999902200227,
 'f_2': 1.9999999902200103,
 'f_3': 1.9999999902199885}

In [3]:
def df_from_responses(all_resp, k):
    # TODO: make the f_1's come from the k
    dd = {"step_n":[], "Type":[], "f_1":[], "f_2":[], "f_3":[]}
    for resp in all_resp:
        first_iter_gid = all_resp[0].group_improvement_direction
        first_iter_grp = all_resp[0].group_reference_point
        for q in range(1,DMs+1): # DM määrä on q
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"DM{q}")
            dd["f_1"].append(resp.reference_points[f"DM{q}"]["f_1"])
            dd["f_2"].append(resp.reference_points[f"DM{q}"]["f_2"])
            dd["f_3"].append(resp.reference_points[f"DM{q}"]["f_3"])
        # add navigation point
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"NAVPOINT")
        dd["f_1"].append(resp.navigation_point["f_1"])
        dd["f_2"].append(resp.navigation_point["f_2"])
        dd["f_3"].append(resp.navigation_point["f_3"])
        # add reachable solution
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"REACHSOL")
        dd["f_1"].append(resp.reachable_solution["f_1"])
        dd["f_2"].append(resp.reachable_solution["f_2"])
        dd["f_3"].append(resp.reachable_solution["f_3"])
        # add group improvement direction
        if resp.group_improvement_direction == None:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GID")
            dd["f_1"].append(first_iter_gid["f_1"])
            dd["f_2"].append(first_iter_gid["f_2"])
            dd["f_3"].append(first_iter_gid["f_3"])
        else:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GID")
            dd["f_1"].append(resp.group_improvement_direction["f_1"])
            dd["f_2"].append(resp.group_improvement_direction["f_2"])
            dd["f_3"].append(resp.group_improvement_direction["f_3"])
        if resp.group_reference_point == None:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GRP")
            dd["f_1"].append(first_iter_grp["f_1"])
            dd["f_2"].append(first_iter_grp["f_2"])
            dd["f_3"].append(first_iter_grp["f_3"])
        else:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GRP")
            dd["f_1"].append(resp.group_reference_point["f_1"])
            dd["f_2"].append(resp.group_reference_point["f_2"])
            dd["f_3"].append(resp.group_reference_point["f_3"])

    ttt = pd.DataFrame(dd)
    return ttt

In [21]:
dd = {"step_n":[], "Type":[], "f_1":[], "f_2":[], "f_3":[]}
for resp in all_resp:
    first_iter_gid = all_resp[0].group_improvement_direction
    first_iter_grp = all_resp[0].group_reference_point
    for q in range(1,4): # DM määrä on q
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"DM{q}")
        dd["f_1"].append(resp.reference_points[f"DM{q}"]["f_1"])
        dd["f_2"].append(resp.reference_points[f"DM{q}"]["f_2"])
        dd["f_3"].append(resp.reference_points[f"DM{q}"]["f_3"])
    # add navigation point
    dd["step_n"].append(resp.step_number)
    dd["Type"].append(f"NAVPOINT")
    dd["f_1"].append(resp.navigation_point["f_1"])
    dd["f_2"].append(resp.navigation_point["f_2"])
    dd["f_3"].append(resp.navigation_point["f_3"])
    # add reachable solution
    dd["step_n"].append(resp.step_number)
    dd["Type"].append(f"REACHSOL")
    dd["f_1"].append(resp.reachable_solution["f_1"])
    dd["f_2"].append(resp.reachable_solution["f_2"])
    dd["f_3"].append(resp.reachable_solution["f_3"])
    # add group improvement direction
    if resp.group_improvement_direction == None:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GID")
        dd["f_1"].append(first_iter_gid["f_1"])
        dd["f_2"].append(first_iter_gid["f_2"])
        dd["f_3"].append(first_iter_gid["f_3"])
    else:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GID")
        dd["f_1"].append(resp.group_improvement_direction["f_1"])
        dd["f_2"].append(resp.group_improvement_direction["f_2"])
        dd["f_3"].append(resp.group_improvement_direction["f_3"])
    if resp.group_reference_point == None:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GRP")
        dd["f_1"].append(first_iter_grp["f_1"])
        dd["f_2"].append(first_iter_grp["f_2"])
        dd["f_3"].append(first_iter_grp["f_3"])
    else:
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"GRP")
        dd["f_1"].append(resp.group_reference_point["f_1"])
        dd["f_2"].append(resp.group_reference_point["f_2"])
        dd["f_3"].append(resp.group_reference_point["f_3"])
       # resp.reference_points["DM1"]["f_1"]
    #dd[]    , resp.navigation_point["f_1"])
    #break
#dd

In [19]:
ttt = pd.DataFrame(dd)
##ttt

,step_n,Type,f_1,f_2,f_3
0,1,DM1,0.500000,0.800000,0.800000
1,1,DM2,0.500000,0.800000,0.800000
2,1,DM3,0.800000,0.800000,0.500000
3,1,NAVPOINT,1.700653,1.741099,1.721087
4,1,REACHSOL,0.503264,0.705494,0.605437
5,1,GID,1.397134,1.200000,1.302866
6,1,GRP,0.602866,0.800000,0.697134
7,2,DM1,0.500000,0.800000,0.800000
8,2,DM2,0.500000,0.800000,0.800000
9,2,DM3,0.800000,0.800000,0.500000


In [6]:
import plotly.express as ex

df = df_from_responses(all_resp, 3)

visu_ideal = [0,0,0]
visu_nadir = [2,2,2]

# näin helppoa se on kun sen vaan saa oikein !!!
fig3 = ex.scatter_3d(df.sort_values("step_n"),
x="f_1", y="f_2", z="f_3",
color="Type",
#range_x=[visu_ideal[0] - 0.5, visu_nadir[0] + 0.5],
#range_y=[visu_ideal[1] - 0.5, visu_nadir[1] + 0.5],
#range_z=[visu_ideal[2] - 0.5, visu_nadir[2] + 0.5],
range_x=[visu_ideal[0] - 0, visu_nadir[0]],
range_y=[visu_ideal[1] - 0, visu_nadir[1]],
range_z=[visu_ideal[2] - 0, visu_nadir[2]],
width= 1000,
height= 1000,
animation_frame=f"step_n",
)

fig3

In [8]:
fig3.write_html("results/S7maxmincones.html")

In [17]:
def df_from_responses2d(all_resp, k):
    # TODO: make the f_1's come from the k
    dd = {"step_n":[], "Type":[], "f_1":[], "f_2":[]}
    for resp in all_resp:
        first_iter_gid = all_resp[0].group_improvement_direction
        first_iter_grp = all_resp[0].group_reference_point
        for q in range(1,4): # DM määrä on q
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"DM{q}")
            dd["f_1"].append(resp.reference_points[f"DM{q}"]["f_1"])
            dd["f_2"].append(resp.reference_points[f"DM{q}"]["f_2"])
            #dd["f_3"].append(resp.reference_points[f"DM{q}"]["f_3"])
        # add navigation point
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"NAVPOINT")
        dd["f_1"].append(resp.navigation_point["f_1"])
        dd["f_2"].append(resp.navigation_point["f_2"])
        #dd["f_3"].append(resp.navigation_point["f_3"])
        # add reachable solution
        dd["step_n"].append(resp.step_number)
        dd["Type"].append(f"REACHSOL")
        dd["f_1"].append(resp.reachable_solution["f_1"])
        dd["f_2"].append(resp.reachable_solution["f_2"])
        #dd["f_3"].append(resp.reachable_solution["f_3"])
        # add group improvement direction
        if resp.group_improvement_direction == None:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GID")
            dd["f_1"].append(first_iter_gid["f_1"])
            dd["f_2"].append(first_iter_gid["f_2"])
            #dd["f_3"].append(first_iter_gid["f_3"])
        else:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GID")
            dd["f_1"].append(resp.group_improvement_direction["f_1"])
            dd["f_2"].append(resp.group_improvement_direction["f_2"])
            #dd["f_3"].append(resp.group_improvement_direction["f_3"])
        if resp.group_reference_point == None:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GRP")
            dd["f_1"].append(first_iter_grp["f_1"])
            dd["f_2"].append(first_iter_grp["f_2"])
            #dd["f_3"].append(first_iter_grp["f_3"])
        else:
            dd["step_n"].append(resp.step_number)
            dd["Type"].append(f"GRP")
            dd["f_1"].append(resp.group_reference_point["f_1"])
            dd["f_2"].append(resp.group_reference_point["f_2"])
            #dd["f_3"].append(resp.group_reference_point["f_3"])

    ttt = pd.DataFrame(dd)
    return ttt

In [18]:
import plotly.express as ex

df = df_from_responses2d(all_resp, 3)

visu_ideal = [0,0]
visu_nadir = [140,50]

# näin helppoa se on kun sen vaan saa oikein !!!
fig2d = ex.scatter(df.sort_values("step_n"),
x="f_1", y="f_2", #z="f_3",
color="Type",
#range_x=[visu_ideal[0] - 0.5, visu_nadir[0] + 0.5],
#range_y=[visu_ideal[1] - 0.5, visu_nadir[1] + 0.5],
#range_z=[visu_ideal[2] - 0.5, visu_nadir[2] + 0.5],
range_x=[visu_ideal[0] - 0, visu_nadir[0]],
range_y=[visu_ideal[1] - 0, visu_nadir[1]],
#range_z=[visu_ideal[2] - 0, visu_nadir[2]],
width= 1000,
height= 1000,
animation_frame=f"step_n",
)

fig2d

In [19]:
fig2d.write_html("results/B1cones.html")

In [ ]:
# does not work but it is fine
fig4 = ex.parallel_coordinates(ttt, color="Type", 
dimensions=["step_n", "f_1", "f_2", "f_3"],
#labels={"step_n": "step", "f_1":"f_1", "f_2":"f_2", "f_3":"f_3", "Type": "type" },
  color_continuous_scale=ex.colors.diverging.Tealrose,
                             color_continuous_midpoint=2)
fig4

In [ ]:
def visu(problem, all_resp):
    lower_bounds = pl.DataFrame(
        [response.reachable_bounds["lower_bounds"] for response in all_resp]
    )
    upper_bounds = pl.DataFrame(
        [response.reachable_bounds["upper_bounds"] for response in all_resp]
    )
    reference_points = pl.DataFrame(
        [response.reference_points for response in all_resp[1:]]
    )
    navigation_points = pl.DataFrame(
        [response.navigation_point for response in all_resp]
    )
    reachable_sols = pl.DataFrame(
        [response.reachable_solution for response in all_resp]
    )
    group_dirs = pl.DataFrame(
        [all_resp[0].group_improvement_direction]
    )

    """
    rps = [] 
    for dm in reference_points:
        max_multiplier = [-1 if obj.maximize else 1 for obj in problem.objectives]
        rp = (
            np.array([reference_points[dm][obj.symbol] for obj in problem.objectives]) * max_multiplier
        )
        rps.append(rp)
    #rps
    """

    import plotly.express as ex

    #iteration_points = pd.DataFrame(
    #    navigation_points,
    #    columns=["f1", "f2", "f3"]
    #)
    fig2 = ex.scatter_3d(
        x=navigation_points[navigation_points.columns[0]],
        y=navigation_points[navigation_points.columns[1]],
        z=navigation_points[navigation_points.columns[2]],
    )
    # TODO: use brain to do this
    # TODO: add RPs
    # TODO: add PF?
    # TODO: add the reachable solutions solution

    #prefs = np.array(
    #    [pref[1] for pref in rps.items()]
    #)
    #prefs = np.vstack((prefs, prefs[0]))
    fig2.add_scatter3d(
        x=[0.8, 0.7, 0.6], 
        y= [0.7, 0.8, 0.5],
        z =[0.5, 0.6, 0.8],)
    #prefs = [
    #    [0.8, 0.7, 0.6],
    #    [0.7, 0.8, 0.5],
    #    [0.5, 0.6, 0.8],
    #]

    #fig2.add_scatter3d(
    #    x=prefs[:, 0],
    #    y=prefs[:, 1],
    #    z=prefs[:, 2],
    #)

    fig2.add_scatter3d(
        x=[group_dirs[0]],
        y=[group_dirs[1]],
        z=[group_dirs[2]],
    )
    fig2.show()



In [ ]:
    "DM1": {"f_1": 0.1, "f_2": 0.4, "f_3": 0.7}, # dm1 and dm2 prefer f1
    "DM2": {"f_1": 0.2, "f_2": 0.2, "f_3": 0.8},
    "DM3": {"f_1": 0.5, "f_2": 0.7, "f_3": 0.3},
    "DM4": {"f_1": 0.3, "f_2": 0.3, "f_3": 0.5},
    "DM5": {"f_1": 0.2, "f_2": 0.5, "f_3": 0.5},

In [20]:
ll = [0.1+0.2+0.5+0.3+0.2,
0.4 +
0.2+
0.7+
0.3+
0.5, 
0.7 +
0.8 +
0.3 +
0.5 +
0.5
 ]
ll

[1.3, 2.1, 2.8]

In [22]:
[ll[0]/5, ll[1]/5, ll[2]/5]

[0.26, 0.42000000000000004, 0.5599999999999999]

In [6]:
#test_dtzl2("mean")
#test_dtzl2("maxmin")
#test_dtzl2("maxmin_cones")
problem = dtlz2(10,3)
all_resp = solve_prob(problem, "maxmin")
visu(problem, all_resp)

maxmin_cones
`xtol` termination condition is satisfied.
Number of iterations: 336, function evaluations: 1680, CG iterations: 0, optimality: 1.96e-01, constraint violation: 1.00e+00, execution time: 0.27 s.
[8.00700745e-09 8.00738320e-09 8.00745991e-09]


In [4]:
ideal = get_ideal_dict(problem)
len(ideal)

3